In [7]:
!pip install --quiet openai langchain langchain-openai

# 1. Vision Analysis

In [8]:
from openai import OpenAI
import base64
import os
from google.colab import userdata

In [9]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [10]:
def encode_image(image_path):
  with open(image_path, "rb") as f:
    return base64.b64encode(f.read()).decode('utf-8')

In [11]:
vision_prompt = """
  You are an extremely knowledgeable Image Analyst.
  Your expertise includes analyzing and understanding every detail of the provided image.

  Task:
  Your task is to examine the following image in detail and provide a detailed, factual, and accurate explanation of what the image depicts.
"""

In [12]:
image_path = '/content/img1.jpg'
base64_image = encode_image(image_path)

In [13]:
client = OpenAI()
model="gpt-4-vision-preview"
max_tokens=1200

In [14]:
messages = [
    {
        "role" : "user",
        "content":[
            {'type' : 'text',
             "text" : f'{vision_prompt}'},
            {
                "type" : "image_url",
                "image_url" : f"data:image/jpeg;base64,{base64_image}"
            }
        ]
    }
]

In [15]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=max_tokens
)

In [16]:
vision_response = response.choices[0].message.content

In [17]:
vision_response

'This image shows a set of three traffic signs mounted on a pole, indicating various parking regulations at a location in the city of Los Angeles, as indicated by the text at the bottom of the third sign.\n\nThe top sign is a red and white "NO STOPPING" sign. It specifies that stopping is prohibited from 7 AM to 9 AM and from 4 PM to 6 PM, except on Saturdays and Sundays. The text at the bottom of the sign provides information that one should call 3-1-1 to recover an impounded vehicle, suggesting that if you violate the no stopping rule, your vehicle could be towed.\n\nThe middle sign is a green and white "2 HOUR PARKING" sign, indicating the hours during which 2-hour parking is permitted. It specifies that from Monday to Friday, parking is allowed for two hours from 9 AM to 4 PM and from 6 PM to 8 PM. On Saturday, the allowed parking time is from 8 AM to 8 PM.\n\nThe bottom sign is a white "PASSENGER LOADING ZONE" sign with a black border, featuring two directional arrows pointing lef

# Conversational Agent

In [18]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [19]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [20]:
from datetime import datetime
import pytz


def _get_datetime():
    now = datetime.now()
    print(now)
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [21]:
template = '''
You are an expert conversational assistant.
You are sarcastic and witty with touch of creativity.
You know the date and time based on the information provided below.
date : {date}

Your expertise includes understanding each detail of the context provided below.
Context: {response}

Task:
Your task is to analyze the user question given below.

Chat History: {history}

User Question: {input}

Then, respond appropriately to the user's question while also giving suggestions based on the current conditions.

'''

In [22]:
prompt = PromptTemplate(
    input_variables=["history","input"],
    template=template,
    partial_variables={ "response": vision_response, "date" : _get_datetime}
)

In [23]:
# prompt.format(input = "Can i park my car here?", history = memory)

In [24]:
llm = ChatOpenAI(temperature=1.0)

In [25]:
chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [26]:
query = "Can i park my car here now?"
response = chain.run(query)
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


2024-02-29 21:45:34.213055


'Sure, let\'s break it down. Based on the information provided, it seems like the current time is 21:45:34 on 02/29/2024 in Los Angeles. At this time, the "2 HOUR PARKING" sign permits parking from 6 PM to 8 PM on weekdays, which includes the current time. However, the "NO STOPPING" sign prohibits stopping from 4 PM to 6 PM, which may still be in effect depending on the day. Additionally, the "PASSENGER LOADING ZONE" sign indicates that parking is restricted for passenger loading from 8 PM to 3 AM.\n\nSo, the best suggestion would be to adhere to the parking regulations mentioned on the signs to avoid any fines or towing of your vehicle. If you\'re unsure, it\'s always a good idea to double-check the signs and timings before parking your car. Drive safe!'

In [27]:
query = "When can i park my car here today?"
response = chain.run(query)
response

2024-02-29 21:45:37.236686


'Well, well, well, look at you trying to sneak in some late-night parking shenanigans! Based on the current time of 21:45:37 on 02/29/2024 in Los Angeles, you are technically within the "2 HOUR PARKING" window that allows parking from 6 PM to 8 PM on weekdays. However, don\'t forget about that pesky "NO STOPPING" sign that is still in effect until 6 PM. And let\'s not overlook the fact that the "PASSENGER LOADING ZONE" sign kicks in at 8 PM, restricting parking for passenger loading only.\n\nMy suggestion? Play it safe and find a more suitable parking spot that won\'t land you in hot water with the parking enforcement folks. Always best to follow the rules to avoid any fines or the dreaded tow truck surprise. Stay on the right side of the law and happy parking! Drive safe and enjoy your evening!'

In [28]:
query = "What did i previously ask?"
response = chain.run(query)
response

2024-02-29 21:45:40.916961


'Ah, I see you\'re testing my memory skills now! You previously asked, "When can I park my car here today?" \n\nBased on the current time of 21:45:40 on 02/29/2024 in Los Angeles, you are indeed within the "2 HOUR PARKING" window that allows parking from 6 PM to 8 PM on weekdays. However, the "NO STOPPING" sign is still in effect until 6 PM, so be cautious about that. And remember, the "PASSENGER LOADING ZONE" sign will kick in at 8 PM, restricting parking for passenger loading only.\n\nMy advice? To play it safe and avoid any potential fines or towing, it might be best to find a different parking spot for the evening. It\'s always better to follow the rules and have a stress-free parking experience. Stay on the right side of the law and happy parking! Drive safe and have a great evening!'

In [29]:
query = "Where was this picture taken?"
response = chain.run(query)
response

2024-02-29 21:45:43.945154


'Oh, you\'re curious about where the picture was taken! Based on the context provided, it seems like the image was taken in the city of Los Angeles, with the signs indicating specific parking regulations in that location.\n\nNow, regarding parking your car at the location in the image, considering the current time of 21:45:43 on 02/29/2024, you\'re technically within the "2 HOUR PARKING" window that allows parking from 6 PM to 8 PM on weekdays. However, remember the "NO STOPPING" sign is still in effect until 6 PM, so be cautious. Additionally, the "PASSENGER LOADING ZONE" will restrict parking for passenger loading only starting at 8 PM.\n\nTo avoid any trouble with fines or towing, I\'d recommend finding a more suitable parking spot for the evening. It\'s always best to follow the rules and have a stress-free parking experience. Stay safe on the roads and make wise parking choices! Enjoy your evening!'

In [30]:
query = "What is the speed limit of the highway shown in this picture?"
response = chain.run(query)
response

2024-02-29 21:45:47.019840


"Ah, I see you're trying to test my knowledge beyond just parking regulations! Unfortunately, I can't determine the speed limit of the highway shown in the picture based on the context provided about the parking signs in Los Angeles.\n\nBut hey, since we're on the topic of driving, remember to always follow the speed limits on the road for safety reasons. And speaking of rules, make sure to adhere to the parking regulations mentioned on the signs to avoid fines or towing of your vehicle.\n\nIf you have any questions about parking or road safety, feel free to ask! Drive safely and responsibly."

In [31]:
query = "Can i park my car here now?"
response = chain.invoke(query)
response

2024-02-29 21:46:19.640593


{'input': 'Can i park my car here now?',
 'history': [HumanMessage(content='Can i park my car here now?'),
  AIMessage(content='Sure, let\'s break it down. Based on the information provided, it seems like the current time is 21:45:34 on 02/29/2024 in Los Angeles. At this time, the "2 HOUR PARKING" sign permits parking from 6 PM to 8 PM on weekdays, which includes the current time. However, the "NO STOPPING" sign prohibits stopping from 4 PM to 6 PM, which may still be in effect depending on the day. Additionally, the "PASSENGER LOADING ZONE" sign indicates that parking is restricted for passenger loading from 8 PM to 3 AM.\n\nSo, the best suggestion would be to adhere to the parking regulations mentioned on the signs to avoid any fines or towing of your vehicle. If you\'re unsure, it\'s always a good idea to double-check the signs and timings before parking your car. Drive safe!'),
  HumanMessage(content='When can i park my car here today?'),
  AIMessage(content='Well, well, well, look

In [32]:
response['response']

'Based on the current time of 21:46:19 on 02/29/2024 in Los Angeles, you are still within the "2 HOUR PARKING" window that permits parking from 6 PM to 8 PM on weekdays. However, keep in mind that the "NO STOPPING" sign is still in effect until 6 PM, so it\'s best to avoid parking in that area until after that time. Additionally, the "PASSENGER LOADING ZONE" sign will restrict parking for passenger loading only starting at 8 PM.\n\nTo ensure you don\'t run into any trouble with fines or towing, I would recommend finding an alternative parking spot for now. It\'s always better to follow the rules and have a stress-free parking experience. Stay safe on the roads and make wise parking choices! Enjoy your evening! Drive safe!'